# 03 - 模型训练模块

此 notebook 负责使用生成的特征数据训练 XGBoost 分类模型，
并对模型性能进行评估和优化。

In [2]:
!pip install matplotlib seaborn joblib xgboost scikit-learn

Looking in indexes: https://pypi.org/simple/
  Using cached scikit_learn-1.8.0-cp313-cp313-macosx_10_13_x86_64.whl.metadata (11 kB)
  Using cached scikit_learn-1.8.0-cp313-cp313-macosx_10_13_x86_64.whl.metadata (11 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.8.0-cp313-cp313-macosx_10_13_x86_64.whl (8.5 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Using cached scikit_learn-1.8.0-cp313-cp313-macosx_10_13_x86_64.whl (8.5 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [scikit-learn] [scikit-learn]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [scikit-learn] [scikit-learn]


In [3]:
import sys
from pathlib import Path
import logging
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# 配置绘图样式
plt.style.use('seaborn-v0_8')
%matplotlib inline

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

# 正确的路径设置
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"\n项目根目录: {project_root}")

print("模型训练环境设置完成")


项目根目录: /Users/anthony/Documents/github/technial_analysis_helper
模型训练环境设置完成


In [5]:
# 加载训练数据
print("正在加载训练数据...")

try:
    # 尝试从 CSV 文件加载（如果之前已生成）
    features_df = pd.read_csv('training_features.csv')
    targets_series = pd.read_csv('training_targets.csv', header=None, names=['target'])['target']
    
    print(f"✓ 从 CSV 文件成功加载训练数据")
    print(f"  特征矩阵形状: {features_df.shape}")
    print(f"  目标向量长度: {len(targets_series)}")
    
except FileNotFoundError:
    print("CSV 文件不存在，尝试从数据库重新生成数据...")
    
    # 从数据库加载并生成数据
    from src.data.mongodb_handler import mongo_handler
    from src.utils.feature_engineering import feature_engineer
    from src.config.settings import config
    
    try:
        if mongo_handler.connect():
            # 加载足够多的数据
            required_records = config.FEATURE_WINDOW_SIZE + 1000  # 额外缓冲
            db_data = mongo_handler.get_candlestick_data(limit=required_records)
            
            if db_data:
                print(f"✓ 从数据库加载 {len(db_data)} 条记录")
                
                # 转换数据格式
                df_raw = pd.DataFrame(db_data)
                df_raw = df_raw.sort_values('timestamp').reset_index(drop=True)
                data_list = df_raw.to_dict('records')
                
                # 生成训练数据
                features_df, targets_series = feature_engineer.create_training_dataset(
                    data_list,
                    stride=10,
                    prediction_horizon=24
                )
                
                if not features_df.empty:
                    print(f"✓ 成功生成训练数据: {features_df.shape}")
                    
                    # 保存为 CSV 供后续使用
                    features_df.to_csv('training_features.csv', index=False)
                    targets_series.to_csv('training_targets.csv', index=False, header=True)
                    print("✓ 数据已保存为 CSV 文件")
                else:
                    print("✗ 无法生成训练数据")
            else:
                print("✗ 数据库中没有足够数据")
                
        mongo_handler.close()
        
    except Exception as e:
        print(f"✗ 加载数据时出错: {e}")

# 检查数据状态
if 'features_df' in locals() and not features_df.empty:
    print(f"\n数据概览:")
    print(f"  样本数: {len(features_df):,}")
    print(f"  特征数: {len(features_df.columns)}")
    print(f"  缺失值: {features_df.isnull().sum().sum():,}")

正在加载训练数据...
✓ 从 CSV 文件成功加载训练数据
  特征矩阵形状: (5275, 155)
  目标向量长度: 5276

数据概览:
  样本数: 5,275
  特征数: 155
  缺失值: 0


In [6]:
# 数据探索性分析
print("正在进行数据探索性分析...")

if 'features_df' in locals() and not features_df.empty:
    # 基本统计信息
    print("\n=== 数据基本统计 ===")
    print(f"数据形状: {features_df.shape}")
    print(f"数值型特征数: {len(features_df.select_dtypes(include=[np.number]).columns)}")
    print(f"非数值型特征数: {len(features_df.select_dtypes(exclude=[np.number]).columns)}")
    
    # 目标变量分布
    print(f"\n=== 目标变量分布 ===")
    target_dist = targets_series.value_counts().sort_index()
    for label, count in target_dist.items():
        percentage = (count / len(targets_series)) * 100
        print(f"类别 {label}: {count:4d} 样本 ({percentage:5.1f}%)")
    
    # 缺失值分析
    missing_data = features_df.isnull().sum()
    missing_percent = (missing_data / len(features_df)) * 100
    missing_summary = pd.DataFrame({
        '缺失数量': missing_data,
        '缺失比例(%)': missing_percent
    }).sort_values('缺失比例(%)', ascending=False)
    
    print(f"\n=== 缺失值最多的前10个特征 ===")
    print(missing_summary.head(10))
    
    # 特征类型分析
    print(f"\n=== 特征类型分析 ===")
    feature_categories = {
        '技术指标(RSI)': [col for col in features_df.columns if 'rsi' in col.lower()],
        '布林带指标': [col for col in features_df.columns if 'bb_' in col.lower()],
        '价格特征': [col for col in features_df.columns if any(prefix in col for prefix in 
                     ['current_', 'price_', 'volume_', 'volatility'])],
        '原始价格序列': [col for col in features_df.columns if col.startswith('raw_')],
        '其他': [col for col in features_df.columns if col == 'timestamp']
    }
    
    for category, cols in feature_categories.items():
        if cols:
            print(f"{category}: {len(cols)} 个特征")
    
    # 数值特征的基本统计
    numeric_features = features_df.select_dtypes(include=[np.number])
    print(f"\n=== 数值特征描述性统计 ===")
    print(numeric_features.describe().round(4))

正在进行数据探索性分析...

=== 数据基本统计 ===
数据形状: (5275, 155)
数值型特征数: 155
非数值型特征数: 0

=== 目标变量分布 ===
类别 -1: 1118 样本 ( 21.2%)
类别 -2:  805 样本 ( 15.3%)
类别 -3:  390 样本 (  7.4%)
类别 0:  882 样本 ( 16.7%)
类别 1: 1051 样本 ( 19.9%)
类别 2:  640 样本 ( 12.1%)
类别 3:  389 样本 (  7.4%)
类别 target:    1 样本 (  0.0%)

=== 缺失值最多的前10个特征 ===
                      缺失数量  缺失比例(%)
short_rsi                0      0.0
short_bb_upper           0      0.0
short_bb_lower           0      0.0
short_bb_middle          0      0.0
short_bb_position        0      0.0
short_price_ma_ratio     0      0.0
medium_rsi               0      0.0
medium_bb_upper          0      0.0
medium_bb_lower          0      0.0
medium_bb_middle         0      0.0

=== 特征类型分析 ===
技术指标(RSI): 3 个特征
布林带指标: 12 个特征
价格特征: 42 个特征
原始价格序列: 131 个特征
其他: 1 个特征

=== 数值特征描述性统计 ===
       short_rsi  short_bb_upper  short_bb_lower  short_bb_middle  \
count  5275.0000       5275.0000       5275.0000        5275.0000   
mean     51.0347       2221.4350       2119.6219        217

In [7]:
# 数据预处理
print("正在进行数据预处理...")

if 'features_df' in locals() and not features_df.empty:
    # 处理缺失值
    print("处理缺失值...")
    initial_missing = features_df.isnull().sum().sum()
    
    # 对于数值型特征，用中位数填充
    numeric_columns = features_df.select_dtypes(include=[np.number]).columns
    features_processed = features_df.copy()
    
    for col in numeric_columns:
        if features_processed[col].isnull().any():
            median_value = features_processed[col].median()
            features_processed[col].fillna(median_value, inplace=True)
    
    # 移除仍含有缺失值的列（通常是字符串或其他类型）
    features_processed = features_processed.dropna(axis=1)
    
    final_missing = features_processed.isnull().sum().sum()
    print(f"  填充前缺失值: {initial_missing:,}")
    print(f"  处理后缺失值: {final_missing:,}")
    
    # 移除时间戳列（通常不用于训练）
    if 'timestamp' in features_processed.columns:
        features_processed = features_processed.drop('timestamp', axis=1)
        print("  已移除 timestamp 列")
    
    print(f"\n预处理后的数据形状: {features_processed.shape}")
    
    # 检查是否有常数特征（方差为0）
    constant_features = []
    for col in features_processed.select_dtypes(include=[np.number]).columns:
        if features_processed[col].var() == 0:
            constant_features.append(col)
    
    if constant_features:
        print(f"\n发现 {len(constant_features)} 个常数特征，正在移除...")
        features_processed = features_processed.drop(columns=constant_features)
        print(f"移除后数据形状: {features_processed.shape}")
    
    # 保存预处理后的数据
    features_processed.to_csv('processed_features.csv', index=False)
    targets_series.to_csv('processed_targets.csv', index=False, header=True)
    print("✓ 预处理后的数据已保存")

正在进行数据预处理...
处理缺失值...
  填充前缺失值: 0
  处理后缺失值: 0
  已移除 timestamp 列

预处理后的数据形状: (5275, 154)
✓ 预处理后的数据已保存
✓ 预处理后的数据已保存


In [8]:
# 模型训练
from src.models.xgboost_trainer import xgb_trainer
from sklearn.model_selection import train_test_split

print("开始模型训练...")

if 'features_processed' in locals() and not features_processed.empty:
    try:
        # 分割训练集和测试集
        X_train, X_test, y_train, y_test = train_test_split(
            features_processed, targets_series, 
            test_size=0.2, 
            random_state=42, 
            stratify=targets_series
        )
        
        print(f"\n数据分割完成:")
        print(f"  训练集: {X_train.shape[0]:,} 样本")
        print(f"  测试集: {X_test.shape[0]:,} 样本")
        print(f"  特征数: {X_train.shape[1]}")
        
        # 训练模型
        print(f"\n正在训练 XGBoost 模型...")
        results = xgb_trainer.train_model(X_train, y_train)
        
        print(f"\n=== 训练结果 ===")
        print(f"训练准确率: {results['accuracy']:.4f}")
        print(f"交叉验证准确率: {results['cv_mean_accuracy']:.4f} ± {results['cv_std_accuracy']:.4f}")
        
        # 显示各类别置信度
        print(f"\n各类别置信度:")
        for class_label, confidence in results['class_confidence'].items():
            print(f"  类别 {class_label}: {confidence:.4f}")
            
        # 在测试集上评估
        print(f"\n在测试集上评估模型...")
        test_predictions, test_probabilities = xgb_trainer.predict(X_test)
        
        from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
        
        test_accuracy = accuracy_score(y_test, test_predictions)
        print(f"测试集准确率: {test_accuracy:.4f}")
        
        print(f"\n分类报告:")
        print(classification_report(y_test, test_predictions, digits=4))
        
    except Exception as e:
        print(f"✗ 模型训练时出错: {e}")
else:
    print("没有可用的预处理数据")

开始模型训练...
✗ 模型训练时出错: Found input variables with inconsistent numbers of samples: [5275, 5276]


In [ ]:
# 模型详细评估
print("正在进行详细的模型评估...")

if 'test_predictions' in locals():
    try:
        # 混淆矩阵
        cm = confusion_matrix(y_test, test_predictions)
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=sorted(y_test.unique()), 
                   yticklabels=sorted(y_test.unique()))
        plt.title('混淆矩阵')
        plt.ylabel('真实标签')
        plt.xlabel('预测标签')
        plt.show()
        
        # 各类别预测概率分布
        plt.figure(figsize=(12, 6))
        
        # 计算每个真实类别的平均预测概率
        unique_classes = sorted(y_test.unique())
        avg_probs_by_class = {}
        
        for true_class in unique_classes:
            mask = (y_test == true_class)
            if mask.sum() > 0:
                # 获取该类别样本的所有预测概率
                class_probs = test_probabilities[mask]
                # 计算每列的平均值（每个预测类别的平均概率）
                avg_probs_by_class[true_class] = class_probs.mean(axis=0)
        
        # 绘制热力图
        if avg_probs_by_class:
            prob_matrix = np.array([avg_probs_by_class[c] for c in unique_classes])
            
            sns.heatmap(prob_matrix, annot=True, fmt='.3f', cmap='YlOrRd',
                       xticklabels=unique_classes,
                       yticklabels=unique_classes)
            plt.title('各类别平均预测概率矩阵')
            plt.xlabel('预测类别')
            plt.ylabel('真实类别')
            plt.show()
        
        # 特征重要性
        if hasattr(xgb_trainer.model, 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'feature': X_train.columns,
                'importance': xgb_trainer.model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            plt.figure(figsize=(10, 8))
            top_features = feature_importance.head(20)
            sns.barplot(data=top_features, y='feature', x='importance')
            plt.title('Top 20 特征重要性')
            plt.xlabel('重要性')
            plt.tight_layout()
            plt.show()
            
            print(f"\nTop 10 重要特征:")
            for idx, row in feature_importance.head(10).iterrows():
                print(f"  {row['feature']}: {row['importance']:.4f}")
        
    except Exception as e:
        print(f"✗ 模型评估时出错: {e}")

In [ ]:
# 交叉验证详细分析
print("正在进行交叉验证分析...")

if 'results' in locals() and 'cv_scores' in results:
    try:
        cv_scores = results['cv_scores']
        
        print(f"\n=== 交叉验证结果 ===")
        print(f"各折准确率: {[f'{score:.4f}' for score in cv_scores]}")
        print(f"平均准确率: {np.mean(cv_scores):.4f}")
        print(f"标准差: {np.std(cv_scores):.4f}")
        print(f"最小值: {np.min(cv_scores):.4f}")
        print(f"最大值: {np.max(cv_scores):.4f}")
        
        # 绘制交叉验证分数分布
        plt.figure(figsize=(10, 6))
        plt.boxplot(cv_scores, vert=False)
        plt.scatter(cv_scores, [1]*len(cv_scores), alpha=0.6, s=50)
        plt.xlabel('准确率')
        plt.title('交叉验证准确率分布')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        # 各类别交叉验证表现
        if 'cv_class_reports' in results:
            print(f"\n各类别交叉验证平均表现:")
            class_metrics = {}
            
            for report in results['cv_class_reports']:
                for class_label, metrics in report.items():
                    if class_label not in ['accuracy', 'macro avg', 'weighted avg']:
                        if class_label not in class_metrics:
                            class_metrics[class_label] = {'precision': [], 'recall': [], 'f1-score': []}
                        
                        class_metrics[class_label]['precision'].append(metrics['precision'])
                        class_metrics[class_label]['recall'].append(metrics['recall'])
                        class_metrics[class_label]['f1-score'].append(metrics['f1-score'])
            
            # 计算平均值和标准差
            for class_label, metrics in class_metrics.items():
                print(f"\n类别 {class_label}:")
                for metric_name, values in metrics.items():
                    mean_val = np.mean(values)
                    std_val = np.std(values)
                    print(f"  {metric_name}: {mean_val:.4f} ± {std_val:.4f}")
                    
    except Exception as e:
        print(f"✗ 交叉验证分析时出错: {e}")

In [ ]:
# 保存模型
print("正在保存训练好的模型...")

try:
    # 保存模型
    save_success = xgb_trainer.save_model()
    
    if save_success:
        print("✓ 模型保存成功")
        
        # 验证模型加载
        print("验证模型加载...")
        load_success = xgb_trainer.load_model()
        
        if load_success:
            print("✓ 模型加载验证成功")
        else:
            print("✗ 模型加载验证失败")
    else:
        print("✗ 模型保存失败")
        
except Exception as e:
    print(f"✗ 模型保存时出错: {e}")

In [ ]:
# 最终模型总结
print("=== 模型训练总结 ===")

if 'results' in locals():
    print(f"\n📊 模型性能指标:")
    print(f"  训练集准确率: {results['accuracy']:.4f}")
    print(f"  交叉验证准确率: {results['cv_mean_accuracy']:.4f} ± {results['cv_std_accuracy']:.4f}")
    
    if 'test_accuracy' in locals():
        print(f"  测试集准确率: {test_accuracy:.4f}")
    
    print(f"\n📈 训练数据信息:")
    print(f"  训练样本数: {len(X_train):,}")
    print(f"  测试样本数: {len(X_test):,}")
    print(f"  特征数量: {X_train.shape[1]}")
    
    print(f"\n🎯 目标类别分布:")
    for label, count in targets_series.value_counts().sort_index().items():
        percentage = (count / len(targets_series)) * 100
        print(f"  类别 {label}: {count:,} 样本 ({percentage:.1f}%)")
    
    print(f"\n💾 模型文件:")
    from src.config.settings import config
    model_path = Path(config.MODEL_SAVE_PATH)
    if model_path.exists():
        size_mb = model_path.stat().st_size / (1024 * 1024)
        print(f"  模型文件: {model_path}")
        print(f"  文件大小: {size_mb:.2f} MB")
    
    print(f"\n✅ 训练完成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 生成性能建议
    print(f"\n💡 性能建议:")
    
    accuracy = results['accuracy']
    cv_std = results['cv_std_accuracy']
    
    if accuracy >= 0.7:
        print("  ✓ 模型性能良好，可以用于实际预测")
    elif accuracy >= 0.6:
        print("  ⚠️ 模型性能一般，建议收集更多数据或调整特征")
    else:
        print("  ✗ 模型性能较差，建议重新设计特征或调整模型参数")
    
    if cv_std <= 0.05:
        print("  ✓ 模型稳定性良好")
    else:
        print("  ⚠️ 模型可能存在过拟合，建议增加正则化或减少特征")

print(f"\n=== 下一步建议 ===")
print("1. 运行 `04_prediction.ipynb` 进行实时预测")
print("2. 如需改进模型，可以返回特征工程阶段调整特征")
print("3. 定期重新训练模型以适应市场变化")